<a href="https://colab.research.google.com/github/SiraHaruethaipree/NLP-Disasters-BERT-fine-tune-huggingface/blob/main/NLP_Disasters_BERT_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
# torch.cuda.get_device_name()

In [ ]:
import pandas as pd 
import numpy as np 
import re
import string 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/NLP huggingface/NLP Disesters/train (1).csv")
test_set = pd.read_csv("/content/drive/MyDrive/NLP huggingface/NLP Disesters/test (1).csv")

In [ ]:
dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
test_set.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
def clean_text(text) :
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text    

In [ ]:
dataset['clean_text'] = dataset['text'].apply(lambda x:clean_text(x))
test_set["clean_text"] = test_set["text"].apply(lambda x:clean_text(x))

In [ ]:
dataset.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [ ]:
dataset.rename(columns = {"target":"labels","clean_text":"sentence1"}, inplace = True)
test_set.rename(columns = {"clean_text":"sentence1"}, inplace = True)

In [ ]:
dataset.head()

,id,keyword,location,text,labels,sentence1
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [ ]:
test_set.head()

,id,keyword,location,text,sentence1
0,0,NaN,NaN,Just happened a terrible car crash,just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills in china and taiwan


In [ ]:
dataset = dataset[["sentence1","labels"]]
test_set1 = test_set[["sentence1"]]

In [ ]:
dataset.head()

,sentence1,labels
0,our deeds are the reason of this earthquake ma...,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are be...,1
3,people receive wildfires evacuation orders in...,1
4,just got sent this photo from ruby alaska as s...,1


In [ ]:
test_set1

,sentence1
0,just happened a terrible car crash
1,heard about earthquake is different cities sta...
2,there is a forest fire at spot pond geese are ...
3,apocalypse lighting spokane wildfires
4,typhoon soudelor kills in china and taiwan
...,...
3258,earthquake safety los angeles ûò safety faste...
3259,storm in ri worse than last hurricane my hard...
3260,green line derailment in chicago
3261,meg issues hazardous weather outlook hwo


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.8 MB/s 
     |████████████████████████████████| 69 kB 7.5 MB/s 
     |████████████████████████████████| 4.7 MB 41.0 MB/s 
     |████████████████████████████████| 120 kB 38.8 MB/s 
     |████████████████████████████████| 212 kB 44.5 MB/s 
     |████████████████████████████████| 115 kB 48.1 MB/s 
     |████████████████████████████████| 127 kB 48.8 MB/s 
     |████████████████████████████████| 6.6 MB 49.6 MB/s 
     |████████████████████████████████| 1.3 MB 45.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 143 kB 5.1 MB/s 


In [ ]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(dataset)
test_dataset = Dataset.from_pandas(test_set1)
freq_dataset

Dataset({
    features: ['sentence1', 'labels'],
    num_rows: 7613
})

In [ ]:
test_dataset

Dataset({
    features: ['sentence1'],
    num_rows: 3263
})

## Train Test split

In [ ]:
freq_dataset_clean = freq_dataset.train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
freq_dataset_clean["validation"] = freq_dataset_clean.pop("test")
freq_dataset_clean["test"] = test_dataset
# # Add the "test" set to our `DatasetDict`
# freq_dataset_clean["test"] = freq_dataset["test"]
# freq_dataset_clean

In [ ]:
freq_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'labels'],
        num_rows: 6090
    })
    validation: Dataset({
        features: ['sentence1', 'labels'],
        num_rows: 1523
    })
    test: Dataset({
        features: ['sentence1'],
        num_rows: 3263
    })
})

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# !apt install git-lfs

In [ ]:
import transformers

print(transformers.__version__)

4.21.2


## Tokenize 

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], truncation=True)

In [ ]:
tokenized_datasets = freq_dataset_clean.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 6090
    })
    validation: Dataset({
        features: ['sentence1', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1523
    })
    test: Dataset({
        features: ['sentence1', 'input_ids', 'attention_mask'],
        num_rows: 3263
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1"])
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

# Build model

In [ ]:
# !pip install wandb

# import wandb
# wandb.login()

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
# metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
batch_size = 32
model_name = "NLP-Disaters-BERT"

args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="matthews_correlation",
    # push_to_hub=True,
    overwrite_output_dir = True,
    # report_to="wandb",  # enable logging to W&B
    # run_name="NLP-Disasters"  # name of the W&B run (optional)
)

In [ ]:
from evaluate import load
matthews_metric = load("matthews_correlation")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return matthews_metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /root/.cache/huggingf

storing https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
creating metadata file for /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight'

In [ ]:
! pip install optuna
! pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 6.2 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 209 kB 40.0 MB/s 
     |████████████████████████████████| 78 kB 2.7 MB/s 
     |████████████████████████████████| 49 kB 430 kB/s 
     |████████████████████████████████| 147 kB 15.5 MB/s 
     |████████████████████████████████| 112 kB 36.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=7a0d65a5aa3512625fb5bab50b2f23ac2ef901451cef000639b68736f68515d4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 1.4 MB/s 
     |████████████████████████████████| 8.8 MB 39.6 M

In [ ]:
import time

In [ ]:
start = time.time()
best_run = trainer.hyperparameter_search(n_trials=1, direction="maximize")
end = time.time()
print("The time of execution of above program is :", end-start)

[I 2022-08-28 15:57:14,233] A new study created in memory with name: no-name-935c9083-288a-4547-849d-f4901ed21c0b
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.576848,0.540261
2,No log,0.472213,0.607434
3,0.553900,0.444938,0.610423
4,0.553900,0.435201,0.622133
5,0.553900,0.433049,0.619544


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/run-0/checkpoint-191
Configuration saved in NLP-Disaters-BERT/run-0/checkpoint-191/config.json
Model weights saved in NLP-Disaters-BERT/run-0/checkpoint-191/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/run-0/checkpoint-191/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/run-0/checkpoint-191/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/run-0/checkpoint-382
Configuration saved in NLP-Disaters-BERT/run-0/checkpoint-382/config.json
Model weights saved in NLP-Disaters-BERT/run-0/checkpoint-382/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/run-0/checkpoint-382/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/run-0/checkpoint-382/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 15

The time of execution of above program is : 5645.554004192352


In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2022-08-26 13:18:20,448] A new study created in memory with name: no-name-6820ea1f-879c-49b3-bb81-d56538e2f522
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.471135,0.593293
2,No log,0.430130,0.621034
3,0.491900,0.424102,0.627676


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/run-0/checkpoint-191
Configuration saved in NLP-Disaters-BERT/run-0/checkpoint-191/config.json
Model weights saved in NLP-Disaters-BERT/run-0/checkpoint-191/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/run-0/checkpoint-191/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/run-0/checkpoint-191/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/run-0/checkpoint-382
Configuration saved in NLP-Disaters-BERT/run-0/checkpoint-382/config.json
Model weights saved in NLP-Disaters-BERT/run-0/checkpoint-382/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/run-0/checkpoint-382/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/run-0/checkpoint-382/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 15

eval/loss,█▂▁
eval/matthews_correlation,▁▇█
eval/runtime,█▁▁
eval/samples_per_second,▁██
eval/steps_per_second,▁██
train/epoch,▁▅▇██
train/global_step,▁▄▇██
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


...

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/run-8/checkpoint-191
Configuration saved in NLP-Disaters-BERT/run-8/checkpoint-191/config.json
Model weights saved in NLP-Disaters-BERT/run-8/checkpoint-191/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/run-8/checkpoint-191/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/run-8/checkpoint-191/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/run-8/checkpoint-382
Configuration saved in NLP-Disaters-BERT/run-8/checkpoint-382/config.json
Model weights saved in NLP-Disaters-BERT/run-8/checkpoint-382/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/run-8/checkpoint-382/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/run-8/checkpoint-382/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 15

eval/loss,▁▁█
eval/matthews_correlation,▆█▁
eval/runtime,▄█▁
eval/samples_per_second,▅▁█
eval/steps_per_second,▅▁█
train/epoch,▁▅▇██
train/global_step,▁▄▇██
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.613455,0.473017


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
[I 2022-08-26 13:28:20,980] Trial 9 pruned. 


In [ ]:
best_run

BestRun(run_id='7', objective=0.6723723630650774, hyperparameters={'learning_rate': 2.8492796853452944e-05, 'num_train_epochs': 2, 'seed': 31, 'per_device_train_batch_size': 8})

In [ ]:
best_run.hyperparameters.items()

dict_items([('learning_rate', 2.8492796853452944e-05), ('num_train_epochs', 2), ('seed', 31), ('per_device_train_batch_size', 8)])

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.391373,0.665213
2,No log,0.397842,0.672372


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/checkpoint-191
Configuration saved in NLP-Disaters-BERT/checkpoint-191/config.json
Model weights saved in NLP-Disaters-BERT/checkpoint-191/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/checkpoint-191/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/checkpoint-191/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to NLP-Disaters-BERT/checkpoint-382
Configuration saved in NLP-Disaters-BERT/checkpoint-382/config.json
Model weights saved in NLP-Disaters-BERT/checkpoint-382/pytorch_model.bin
tokenizer config file saved in NLP-Disaters-BERT/checkpoint-382/tokenizer_config.json
Special tokens file saved in NLP-Disaters-BERT/checkpoint-382/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from 

TrainOutput(global_step=382, training_loss=0.3865515823763703, metrics={'train_runtime': 45.1668, 'train_samples_per_second': 269.667, 'train_steps_per_second': 8.458, 'total_flos': 158286122575296.0, 'train_loss': 0.3865515823763703, 'epoch': 2.0})

In [ ]:
trainer.evaluate(tokenized_datasets["validation"])

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32


{'eval_loss': 0.3978423774242401,
 'eval_matthews_correlation': 0.6723723630650774,
 'eval_runtime': 1.3631,
 'eval_samples_per_second': 1117.269,
 'eval_steps_per_second': 35.213,
 'epoch': 2.0}

In [ ]:
# wandb.finish()

eval/loss,█▁▁▁
eval/matthews_correlation,▁███
eval/runtime,▁█▁▁
eval/samples_per_second,█▁██
eval/steps_per_second,█▁██
train/epoch,▁▁███
train/global_step,▁▁███
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


In [ ]:
predict = trainer.predict(tokenized_datasets["test"])

In [ ]:
preds = np.argmax(predict.predictions, axis=-1)

In [ ]:
preds[:20]

In [ ]:
preds.shape

In [ ]:
test_set

In [ ]:
submit_df = test_set[["id"]]
submit_df["target"] = preds

In [ ]:
submit_df.head(10)

In [ ]:
submit_df.set_index("id",inplace = True)

In [ ]:
submit_df.to_csv("bert_submission.csv")